# > New File

In [ ]:
import pandas as pd 
import numpy as np

In [ ]:
d1 = pd.read_csv("/kaggle/input/assignment-reviews-metadata/product_asin.csv",encoding="utf-8")
d2 = pd.read_csv("/kaggle/input/assignment-reviews-metadata/reviews_supplements.csv",encoding="utf-8")

In [ ]:
d1.columns,d2.columns

In [ ]:
d2['hieer_check'] = d2['parent_asin'] == d2['asin']

In [ ]:
d2[d2['hieer_check'] == True].shape,d2[d2['hieer_check'] != True].shape,d2['hieer_check'].shape

In [ ]:
d1['parent_asin'].shape,d1['parent_asin'].unique().size

In [ ]:
!pip install sdv

In [ ]:
import sdv

In [ ]:
dir(sdv.multi_table)

In [ ]:
product_data = pd.read_csv('/kaggle/input/assignment-reviews-metadata/product_asin.csv')
reviews_data = pd.read_csv('/kaggle/input/assignment-reviews-metadata/reviews_supplements.csv')

In [ ]:
import datetime
reviews_data['time'] = reviews_data['time'].apply(lambda x: datetime.datetime.strptime(x, '%H:%M').strftime('%H:%M:%S'))

In [ ]:
from sdv.metadata import Metadata

metadata = Metadata.detect_from_dataframes(
    data={
        'products': product_data,
        'reviews': reviews_data
    })

In [ ]:
# update_column products table columns
metadata.update_column(
    table_name='products',
    column_name='title',
    sdtype='text',
    pii=False
)

metadata.update_column(
    table_name='products',
    column_name='parent_asin',
    sdtype='id',

)

for cat in ['categories', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6']:
    metadata.update_column(
        table_name='products',
        column_name=cat,
        sdtype='categorical'
    )

# update_column reviews table columns
metadata.update_column(
    table_name='reviews',
    column_name='rating',
    sdtype='numerical',
    computer_representation='Int64'
)

metadata.update_column(
    table_name='reviews',
    column_name='title',
    sdtype='text',
    pii=False
)

metadata.update_column(
    table_name='reviews',
    column_name='text',
    sdtype='text',
    pii=False
)

metadata.update_column(
    table_name='reviews',
    column_name='asin',
    sdtype='categorical'
)

metadata.update_column(
    table_name='reviews',
    column_name='parent_asin',
    sdtype='id'
)

metadata.update_column(
    table_name='reviews',
    column_name='user_id',
    sdtype='id',

)

metadata.update_column(
    table_name='reviews',
    column_name='helpful_vote',
    sdtype='numerical',
    computer_representation='Int64'
)

metadata.update_column(
    table_name='reviews',
    column_name='verified_purchase',
    sdtype='boolean'
)

metadata.update_column(
    table_name='reviews',
    column_name='date',
    sdtype='datetime',
    datetime_format='%Y-%m-%d'
)

metadata.update_column(
    table_name='reviews',
    column_name='time',
    sdtype='datetime',
    datetime_format='%H:%M:%S'
)


metadata.set_primary_key(
    column_name='parent_asin',
    table_name='products',
)


# Add relationship between tables
metadata.add_relationship(
    'products',
    'reviews',
    'parent_asin',
    'parent_asin'
)



In [ ]:
metadata.visualize(
    show_table_details='full',
    show_relationship_labels=True,
    output_filepath='my_metadata.png'
)

In [ ]:
metadata.save_to_json(filepath='metadata.json')

In [ ]:
from sdv.metadata import MultiTableMetadata
from sdv.multi_table import HMASynthesizer
import pandas as pd

# Load the datasets



# Create the synthesizer
synthesizer = HMASynthesizer(metadata)

# Fit the synthesizer
synthesizer.fit({
    'products': product_data,
    'reviews': reviews_data
})

# Generate synthetic data
synthetic_data = synthesizer.sample()

# Access synthetic data for each table
synthetic_product_data = synthetic_data['products']
synthetic_reviews_data = synthetic_data['reviews']

# Save the generated synthetic data to CSV files
synthetic_product_data.to_csv('synthetic_product_data.csv', index=False)
synthetic_reviews_data.to_csv('synthetic_reviews_data.csv', index=False)

print("Synthetic data generation completed and saved to CSV files.")